In [1]:
# TODO: investigate and prepare data

In [75]:
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
import re

from skimage.io import imread

In [80]:
DATA_DIR = "./data/nsc_img/img"
TRAIN_SPLIT = "./data/file_list/splits/train.txt"
VAL_SPLIT = "./data/file_list/splits/val.txt"
TEST_SPLIT = "./data/file_list/splits/test.txt"

In [85]:
def get_paths(img_dir):
    ret_dict = {}
    for root, dirs, files in os.walk(img_dir):
        if len(files) != 0:
            for fl in files:
                path = os.path.join(root, fl)
                img_id = "".join(re.findall(r"[0-9]+", root))
                ret_dict[img_id] = path
    return ret_dict

def split_data(paths, train_split_f, val_split_f, test_split_f):
    train, val, test = [], [], []
    for split,split_f in ((train, train_split_f), (val, val_split_f), (test, test_split_f)):
        with open(split_f) as rf:
            for line in rf:
                line = line.split(".")[0]
                split.append(paths[line])
    return train, val, test

In [79]:
paths = get_paths(DATA_DIR)

In [86]:
train, val, test = split_data(paths, TRAIN_SPLIT, VAL_SPLIT, TEST_SPLIT)

In [91]:
print("All data: ", len(paths.keys()))

All data:  355206


In [90]:
print("Train data: {} \nVal data: {} \nTest data: {}".format(
    len(train), len(val), len(test)))

Train data: 28000 
Val data: 1000 
Test data: 1000


In [92]:
train[10]

'./data/nsc_img/img/002/844/823/01.jpg'

In [1]:
test[1]

NameError: name 'test' is not defined

In [96]:
list(paths.items())[10]

('004868273', './data/nsc_img/img/004/868/273/01.jpg')

In [100]:
import pickle

In [97]:
posts_path = "./data/posts_nsc.pickle"
usrs_path = "./data/users_nsc.pickle"

In [101]:
with open(posts_path, "rb") as rf:
    posts_data = pickle.load(rf)

In [102]:
with open(usrs_path, "rb") as rf:
    usrs_data = pickle.load(rf)

In [152]:
comments = posts_data["004868273"]["Comments"]
comments = re.sub(r"https?://", "", comments)
comments = re.sub(r"\"", "", comments)
comments = re.findall(r"\{(.*?)\}", comments)
comm_d = {}
comms = []
new_com = False
for com in comments:
    split = com.split(":")
    if len(split) > 2:
        split = [split[0], ":".join(split[1:])]
    type_, attr = split
    comm_d[type_] = attr.strip()
    if {"Name", "Url", "Comment"} == set(comm_d.keys()):
        comms.append(comm_d)
        comm_d = {}
print(comms)

[{'Name': 'Lucy M.', 'Url': 'lookbook.nu/lucymitchell12', 'Comment': 'The skirt is gorgeous :)'}, {'Name': 'Lo P.', 'Url': 'lookbook.nu/lop', 'Comment': 'so chic and stylish!'}, {'Name': 'GaiL J.', 'Url': 'lookbook.nu/hollahoxz', 'Comment': 'love it!'}, {'Name': 'Poldo  N.', 'Url': 'lookbook.nu/napitpoldo', 'Comment': 'love your skirt!'}, {'Name': 'Meredith J.', 'Url': 'lookbook.nu/merejess', 'Comment': 'Love the color combo of this outfit and the unexpected touch of silver shoes.'}, {'Name': 'Tristian N.', 'Url': 'lookbook.nu/tristiann', 'Comment': 'This is very cute and very San Francisco! perfect!'}, {'Name': 'Hugo S.', 'Url': 'lookbook.nu/hunggofunggo', 'Comment': 'That skirt is to die for omg <3'}, {'Name': 'Christina M.', 'Url': 'lookbook.nu/christina_marie', 'Comment': 'Love it'}, {'Name': 'Ad J.', 'Url': 'lookbook.nu/adj', 'Comment': 'Perfect look! Remarkable colours and textures combo.'}, {'Name': 'Inbal R.', 'Url': 'lookbook.nu/inbalr', 'Comment': 'lovely colors and gorgeous 

In [155]:
list(usrs_data.items())[:3]

[('1482503',
  {'Age': 29,
   'City': 'Scotland',
   'CommentKarma': 0,
   'CommentNum': 2,
   'Country': 'United Kingdom',
   'FanNum': 0,
   'Fans': '[]',
   'Followers': '[]',
   'FollowingNum': 20,
   'HeartNum': 0,
   'Info': '29 歲 女性 來自 Scotland, United Kingdom',
   'Karma': 0,
   'LookNum': 0,
   'Sex': 'Female',
   'TopicNum': 0,
   'UserName': 'Fifi Q'}),
 ('2222424',
  {'Age': 22,
   'City': None,
   'CommentKarma': 0,
   'CommentNum': 57,
   'Country': 'Albania',
   'FanNum': 24,
   'Fans': '[]',
   'Followers': '[]',
   'FollowingNum': 94,
   'HeartNum': 0,
   'Info': '22 歲 女性 來自 Albania',
   'Karma': 5,
   'LookNum': 0,
   'Sex': 'Female',
   'TopicNum': 0,
   'UserName': 'Lool M'}),
 ('1200459',
  {'Age': 24,
   'City': 'Montréal',
   'CommentKarma': 0,
   'CommentNum': 1,
   'Country': 'Canada',
   'FanNum': 2,
   'Fans': '[]',
   'Followers': '[]',
   'FollowingNum': 16,
   'HeartNum': 0,
   'Info': '24 歲 Dancer and fashion design student 來自 Montréal, Canada',
   'Karma

In [156]:
# TODO: check urls, how users with same urls posted comments

In [159]:
import json

In [157]:
comments = posts_data["004868273"]["Comments"]

In [160]:
comments = json.loads(comments)

In [175]:
comments[12][1]

{'Url': 'http://lookbook.nu/corneroftheworld'}

In [180]:
from collections import defaultdict
from tqdm import tqdm_notebook

In [186]:
com_count = defaultdict(int)
for postid in tqdm_notebook(posts_data):
    comments = posts_data[postid]["Comments"]
    try:
        comments = json.loads(comments)
    except:
        continue
    for comm in comments:
        com_count[comm[1]["Url"]]+= 1

In [187]:
com_counts = sorted(com_count.items(), key=lambda x: -x[1])

In [202]:
com_counts[:10]

[('http://lookbook.nu/queenhorsfall', 46947),
 ('http://lookbook.nu/aniazarzycka', 36009),
 ('http://lookbook.nu/lop', 35401),
 ('http://lookbook.nu/beautymarklady', 34131),
 ('http://lookbook.nu/prisciladiniz', 32999),
 ('http://lookbook.nu/tneale', 31338),
 ('http://lookbook.nu/viktoriyasener', 29260),
 ('http://lookbook.nu/ivanakada', 28434),
 ('http://lookbook.nu/dopeboyswag', 23937),
 ('http://lookbook.nu/avalon1982', 23351)]

In [192]:
len(com_counts)

171369

In [198]:
posts_data["004868273"]

{'CommentNum': 27,
 'Comments': '[[{"Name": "Lucy M."}, {"Url": "http://lookbook.nu/lucymitchell12"}, {"Comment": "The skirt is gorgeous :)"}], [{"Name": "Lo P."}, {"Url": "http://lookbook.nu/lop"}, {"Comment": "so chic and stylish!"}], [{"Name": "GaiL J."}, {"Url": "http://lookbook.nu/hollahoxz"}, {"Comment": "love it!"}], [{"Name": "Poldo  N."}, {"Url": "http://lookbook.nu/napitpoldo"}, {"Comment": "love your skirt!"}], [{"Name": "Meredith J."}, {"Url": "http://lookbook.nu/merejess"}, {"Comment": "Love the color combo of this outfit and the unexpected touch of silver shoes."}], [{"Name": "Tristian N."}, {"Url": "http://lookbook.nu/tristiann"}, {"Comment": "This is very cute and very San Francisco! perfect!"}], [{"Name": "Hugo S."}, {"Url": "http://lookbook.nu/hunggofunggo"}, {"Comment": "That skirt is to die for omg <3"}], [{"Name": "Christina M."}, {"Url": "http://lookbook.nu/christina_marie"}, {"Comment": "Love it"}], [{"Name": "Ad J."}, {"Url": "http://lookbook.nu/adj"}, {"Comment

In [1]:
def some_f(*args):
    print(args[1])

In [2]:
some_f(1, 2)

2
